In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn import datasets 
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('hate_speech_data_train.csv') 
df

,class,tweet
0,1,RT @__Junebugg: @VoiceOfDStreetz hell yea save...
1,2,How the hell was David Murphy's hit not a home...
2,1,RT @FeeelGreatness: You don't know where your ...
3,1,Thats some hoe shit doe
4,2,I just want vanilla Oreos
...,...,...
19821,1,RT @davegetnmoney: I beat the pussy up up up u...
19822,2,RT @RT_America: Russell Brand mocks Bill O'Rei...
19823,1,@_B_R_Y_C_E_ what happened to going fishing bi...
19824,2,RT @BriannDominguez: Gasoline - daddy Yankee &...


In [3]:
df.shape

(19826, 2)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19826 entries, 0 to 19825
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   class   19826 non-null  int64 
 1   tweet   19826 non-null  object
dtypes: int64(1), object(1)
memory usage: 309.9+ KB


In [5]:
df.isnull().sum()

class    0
tweet    0
dtype: int64

In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re

nltk.download('stopwords')
port_stem=PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tande\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [7]:
port_stem

<PorterStemmer>

In [8]:
port_stem.stem("Hi thIs is crimson  * % %@@@")

'hi this is crimson  * % %@@@'

In [9]:
def stemming(content):
    con=re.sub('[^a-zA-Z]', ' ', content)
    con=con.lower()
    con=con.split()
    con=[port_stem.stem(word) for word in con if not word in stopwords.words('english')]
    con=' '.join(con)
    return con

In [10]:
df['tweet']= df['tweet'].apply(stemming)

In [11]:
X = df['tweet']
Y = df['class']


In [12]:
X.shape

(19826,)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)

(15860,)
(3966,)


In [27]:
# import pandas as pd
# import numpy as np
# import pickle
# import gensim.downloader as api
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.metrics import classification_report

# # Load the pre-trained Word2Vec model
# model = api.load("word2vec-google-news-300")

# model = pickle.load(open('word2vec_model.pkl', 'rb'))


# # Save the model to a file using pickle
# with open('word2vec_model.pkl', 'wb') as f:
#     pickle.dump(model, f)

import pandas as pd
import numpy as np
import pickle
import gensim.downloader as api
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

# Step 1: Load the pre-trained Word2Vec model using gensim
print("Downloading the Word2Vec model...")
model = api.load("word2vec-google-news-300")
print("Model downloaded successfully!")

# Step 2: Save the model to a file using pickle
with open('word2vec_model.pkl', 'wb') as f:
    pickle.dump(model, f)
print("Model saved to 'word2vec_model.pkl' successfully!")

# Step 3: Load the model from the file (later when needed)
# You can comment out this block if you are just saving the model
with open('word2vec_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)
print("Model loaded from 'word2vec_model.pkl' successfully!")

Model downloaded successfully!
Model saved to 'word2vec_model.pkl' successfully!
Model loaded from 'word2vec_model.pkl' successfully!


In [28]:
import numpy as np 

# Define a function to convert a tweet into a vector using Word2Vec
def tweet_to_vector(tweet):
    words = tweet.split()
    vectors = []
    for word in words:
        try:
            vectors.append(model[word])
        except KeyError:
            # Ignore out-of-vocabulary words
            pass
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# Convert the training and testing tweets to vectors using Word2Vec
x_train = np.array([tweet_to_vector(tweet) for tweet in X_train])
x_test = np.array([tweet_to_vector(tweet) for tweet in X_test])



In [29]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.svm import SVC 

model2=DecisionTreeClassifier()
model2.fit(x_train, y_train)
prediction=model2.predict(x_test)
prediction
model2.score(x_test, y_test)


0.7481089258698941

In [30]:
import pickle

# with open('word2vec_model.pkl', 'wb') as f:
#     pickle.dump(model, f)

with open('model.pkl', 'wb') as f:
    pickle.dump(model2, f)

In [31]:
import pickle

def predict_hate_speech(model_path, tweet):
    # Load the trained model
    with open(model_path, 'rb') as f:
        model2 = pickle.load(f)
    # Convert the tweet to a Word2Vec vector
    vector = tweet_to_vector(tweet)
    # Make the prediction
    prediction = model2.predict(vector.reshape(1, -1))
    if prediction==[0]:
      print('not offensive')
    elif prediction==[1]:
      print('offensive')
    else:
      print('hate')
      

In [32]:
tweet = "Love is in the air"
model_path = "model.pkl"
prediction = predict_hate_speech(model_path, tweet)



hate


In [33]:
tweet = "RT @Talkmaster: People who would vote for Charlie Crist and Michelle Nunn are why our founders did NOT put a right to vote in our Constitut &#8230;"
model_path = "model.pkl"
prediction = predict_hate_speech(model_path, tweet)

hate


In [34]:
tweet = "It is a good day  whatsoever"
model_path = "model.pkl"
prediction = predict_hate_speech(model_path, tweet)

not offensive


In [35]:
tweet = "@NorahODonnell dishing Texas trash talking pts with racist @jdickerson maybe if GBush had apologize for all his sins he wouldn't be hiding and I fucking hate this fucking neighborhood"
model_path = "model.pkl"
prediction = predict_hate_speech(model_path, tweet)



offensive
